# The fermionic tweezer backend

It implements four optical lattice sites with possibility of spin up and down. The first four wires are the spin up and the next four wires are the spin down.

 We have implemented:
 
 - `load` which adds a Fermion to the wire.
 - `hop` which lets Fermions hop.
 - `int` which describes interactions between fermions.
 - `phase` which is the chemical potential on the gate.
 - `measure` which reads out the occupation.

# Our own simulator code

The fermions can be directly mapped to spins via a local Jordan-Wigner transformation on site $s$

$\psi_{x,-1/2} = -\sigma^{+} \otimes \mathbf{1} \otimes \mathbf{1} \otimes \mathbf{1}$

$\psi_{x,1/2} =  -\sigma^z \otimes \sigma^{+} \otimes \mathbf{1} \otimes \mathbf{1}$

and on site $y$ 

$\psi_{y,-1/2} = -\sigma^z \otimes \sigma^z \otimes \sigma^+ \otimes \mathbf{1}$

$\psi_{y,1/2} =  -\sigma^z \otimes \sigma^z \otimes \sigma^z \otimes \sigma^{+} $

We first create the Pauli matrices

and then create the fermionic operators on the extend system

In [2]:
import numpy as np
from scipy.sparse.linalg import expm

In [3]:
def nested_kronecker_product(a):
    '''putting together a large operator from a list of matrices.
    
    Provide an example here.
    
    Args:
        a (list): A list of matrices that can connected.

    Returns:
        array: An matrix that operates on the connected Hilbert space.
    '''
    if len(a) == 2:
        return np.kron(a[0],a[1])
    else:
        return np.kron(a[0], nested_kronecker_product(a[1:]))

In [4]:
def jordan_wigner_transform(j, lattice_length):
    '''
    Builds up the fermionic operators in a 1D lattice
    
    Args:
        j (int): site index
        lattice_length: how many sites does the lattice have ?
    
    Returns:
        psi_x: the field operator of creating a fermion on size j
    '''
    P = np.array([[0, 1], [0, 0]])
    Z = np.array([[1, 0], [0, -1]])
    I = np.eye(2)
    operators = []
    for k in range(j):
        operators.append(Z)
    operators.append(P)
    for k in range(lattice_length-j-1):
        operators.append(I)
    return nested_kronecker_product(operators)

we have basically four wires, which is the same as two lattice sites with spin $\pm 1/2$

In [10]:
l = 2  # length of the tweezer array
Nstates = 2 ** (2 * l)

lattice_length = 2 * l
loweringOp = []
for i in range(lattice_length):
    loweringOp.append(jordan_wigner_transform(i, lattice_length))
    

In [11]:
Nstates = 2**lattice_length

In [13]:
emptySystem = np.zeros(Nstates)
emptySystem[0] = 1
print(emptySystem)

# load one atom into site one
psi0 = loweringOp[1].T.dot(emptySystem)
print(psi0)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


measurement

In [14]:
number_operators = []
for i in range(lattice_length):
    number_operators.append(loweringOp[i].T.conj().dot(loweringOp[i]))

In [15]:
probs = np.abs(psi0)**2
resultInd = np.random.choice(np.arange(Nstates), p=probs, size = 1)

print(resultInd)

result = np.zeros(Nstates)
result[resultInd[0]] = 1
print(result)

measurements = np.zeros(lattice_length, dtype = int)
for i in range(lattice_length):
    observed = number_operators[i].dot(result)
    observed = observed.dot(result)
    measurements[i] = int(observed)
    
print(measurements)

[4]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 1 0 0]


## construct the hopping operator

In [19]:
emptySystem = 1j*np.zeros(Nstates)
emptySystem[0] = 1
print(emptySystem)

# load two atoms into site one
psi0 = loweringOp[0].T.dot(emptySystem)
psi0 = loweringOp[1].T.dot(psi0)

print(psi0)

[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j]


couple two neighboring sites

In [20]:
theta = np.pi/2;
latt_ind = [0, 1, 2, 3];

# couple spin down sites with even indices
Hhop = loweringOp[latt_ind[0]].T.dot(loweringOp[latt_ind[2]]) + loweringOp[latt_ind[2]].T.dot(loweringOp[latt_ind[0]])
# couple spin up sites with odd indices
Hhop += loweringOp[latt_ind[1]].T.dot(loweringOp[latt_ind[3]]) + loweringOp[latt_ind[3]].T.dot(loweringOp[latt_ind[1]])

Uhop = expm(-1j*theta*Hhop); 

In [21]:
psi = np.dot(Uhop,psi0)
psi

array([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  1.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00-1.14293026e-16j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+1.14293026e-16j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
       -5.43731249e-17+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j])

and measure

In [23]:
measurement_indices = [0,1,2,3]
n_shots = 5
probs = np.abs(psi)**2
resultInd = np.random.choice(np.arange(Nstates), p=probs, size = n_shots)

measurements = np.zeros((n_shots, len(measurement_indices)), dtype = int)
for jj in range(n_shots):
    result = np.zeros(Nstates)
    result[resultInd[jj]] = 1

    for ii, ind in enumerate(measurement_indices):
        observed = number_operators[ind].dot(result)
        observed = observed.dot(result)
        measurements[jj,ii] = int(observed)
        
measurements

array([[0, 0, 1, 1],
       [0, 0, 1, 1],
       [0, 0, 1, 1],
       [0, 0, 1, 1],
       [0, 0, 1, 1]])

## interactions

In [24]:
emptySystem = 1j*np.zeros(Nstates)
emptySystem[0] = 1
print(emptySystem)

# load two atoms into site one
psi0 = loweringOp[0].T.dot(emptySystem)
psi0 = loweringOp[1].T.dot(psi0)

print(psi0)

[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j]


In [30]:
number_operators = []
for i in range(lattice_length):
    number_operators.append(loweringOp[i].T.conj().dot(loweringOp[i]))

# interaction Hamiltonian
Hint = 0 * number_operators[0]
for ii in range(l):
    spindown_ind = 2*ii;
    spinup_ind = 2*ii+1;
    Hint += number_operators[spindown_ind].dot(number_operators[spinup_ind])

In [31]:
np.array(number_operators).sum(axis=0)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

why do I have six states with two atoms ?

In [32]:
Hint

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

why would there ever be a two ?